In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from datetime import datetime as dt
import pandas as pd
from db.db_operations import execute_db_operations

from generators.full_generators import (
    create_company_data,
    create_all_erp_data,
    create_mapping_between_all, 
)

company_name = "Ilva"

In [2]:
data = create_company_data(company_name=company_name, save_to_csv=True)

Generating data for company: Ilva...
Context data for Ilva generated: {'company_name': 'Ilva', 'count_employee': 400, 'count_department': 20, 'count_customer': 50, 'count_product': 150, 'count_procurement': 120, 'count_service': 100, 'count_account': 60, 'count_vendor': 60, 'estimated_product': 80000000, 'estimated_service': 20000000, 'estimated_overhead': 15000000, 'estimated_revenue': 1200000000}
Generating dimensions for company: Ilva.
This usually takes 2-5 mins.

=== Attempt 1 ===
✔ Roles and Names generated.
✔ Procurement data generated.
✔ Services data generated.
✔ Products data generated.
✔ Customers data generated.
✔ Departments data generated.
✔ Vendors data generated.
✔ Business Units and company data generated.
✔ Payroll data generated.
✔ All table lengths validated.
✔ All CSVs saved to: data/outputdata

 * Semantic mapping started * :
Time estimate: 3-5 minutes
✔ Procurement mapping done!
✔ Service mapping done!
✔ All mapping data generated.
✔ All mapping CSVs saved to: da

c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\modeling\budget.py:576: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Prisændring', 'Fejlpostering', 'Periodisering', nan, 'Periodisering', nan, 'Prisændring', 'Prisændring', 'Andet', 'Fejlpostering', nan, 'Andet', 'Prisændring', nan, 'Aktivitetsændring', 'Aktivitetsændring', nan, nan, nan, nan, nan, 'Aktivitetsændring', 'Andet', nan, nan, 'Periodisering', 'Fejlpostering', 'Andet', 'Prisændring', 'Aktivitetsændring', 'Fejlpostering', nan, 'Prisændring', 'Aktivitetsændring', nan, nan, 'Periodisering', nan, nan, nan, nan, nan, 'Periodisering', 'Andet', 'Fejlpostering', 'Andet', 'Fejlpostering', 'Aktivitetsændring', nan, 'Periodisering', 'Periodisering', 'Fejlpostering', 'Andet', 'Prisændring', nan, nan, 'Aktivitetsændring', nan, 'Fejlpostering', 'Andet', nan, 'Andet', nan, 'Aktivitetsændring', 'Fejlpostering', nan, nan, nan, nan, n

✔ Reporting data saved and generated!
✔ All ERP data and mapping generated for company: Ilva


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\modeling\budget.py:576: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Prisændring', 'Fejlpostering', 'Periodisering', nan, 'Periodisering', nan, 'Prisændring', 'Prisændring', 'Andet', 'Fejlpostering', nan, 'Andet', 'Prisændring', nan, 'Aktivitetsændring', 'Aktivitetsændring', nan, nan, nan, nan, nan, 'Aktivitetsændring', 'Andet', nan, nan, 'Periodisering', 'Fejlpostering', 'Andet', 'Prisændring', 'Aktivitetsændring', 'Fejlpostering', nan, 'Prisændring', 'Aktivitetsændring', nan, nan, 'Periodisering', nan, nan, nan, nan, nan, 'Periodisering', 'Andet', 'Fejlpostering', 'Andet', 'Fejlpostering', 'Aktivitetsændring', nan, 'Periodisering', 'Periodisering', 'Fejlpostering', 'Andet', 'Prisændring', nan, nan, 'Aktivitetsændring', nan, 'Fejlpostering', 'Andet', nan, 'Andet', nan, 'Aktivitetsændring', 'Fejlpostering', nan, nan, nan, nan, n

In [3]:
now = dt.now()
version_tag = (
    company_name.lower() 
    + now.strftime("%d") + "/" + now.strftime("%m") + ". Kl: " + now.strftime("%H") + ":" + now.strftime("%M")
)
execute_db_operations(version_tag)

[INFO] Inserting 20 rows into dim_department using to_sql...
[INFO] Inserting 59 rows into dim_customer using to_sql...
[INFO] Inserting 150 rows into dim_product using to_sql...
[INFO] Inserting 109 rows into dim_account using to_sql...
[INFO] Inserting 120 rows into dim_procurement using to_sql...
[INFO] Inserting 100 rows into dim_service using to_sql...
[INFO] Inserting 14 rows into dim_line using to_sql...
[INFO] Inserting 69 rows into dim_vendor using to_sql...
[INFO] Inserting 9 rows into dim_bu using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)


[INFO] Inserting 400 rows into dim_employee using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)


[INFO] Inserting 134400 rows into fact_payroll using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:189: DtypeWarning: Columns (0,1,6,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_erp = pd.read_csv("data/outputdata/fact/general_ledger.csv")
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_opti

[INFO] Inserting 232327 rows into fact_general_ledger using to_sql...
[INFO] Inserting 1050 rows into fact_reporting using to_sql...
